In [1]:
import scipy as sc
import numpy as np
!pip install gpxpy
import gpxpy
import requests
import urllib
import pandas as pd
import math
!pip install geopy

In [2]:
# points must have elevation data established to call this method
from geopy import distance
def distanceBetweenPoints(p1, p2):
    flat_distance = distance.distance(p1[:2], p2[:2]).m
    # print(flat_distance)
    euclidian_distance = math.sqrt(flat_distance**2 + (p2[2] - p1[2])**2)
    return euclidian_distance

In [3]:
# coordinates with known elevation 
# these are the coordinates we want to get elevations from
latOne = [48.633, 48.733, 45.1947, 45.1962]
lonOne = [-93.9667, -94.6167, -93.3257, -93.2755]

# create data frame
#df = pd.DataFrame({
 #   'lat': latOne,
  #  'lon': lonOne
#})

In [4]:


# USGS Elevation Point Query Service
url = r'https://nationalmap.gov/epqs/pqs.php?'



def elevation_function(df, lat_column, lon_column):
    """Query service using lat, lon. add the elevation values as a new column."""
    elevations = []
    
    for lat, lon in zip(df[lat_column], df[lon_column]):

        # define rest query params
        params = {
            'output': 'json',
            'x': lon,
            'y': lat,
            'units': 'Meters'
        }

        # format query string and return query value
        result = requests.get((url + urllib.parse.urlencode(params)))
        elevations.append(result.json()['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation'])


    df['elev_meters'] = elevations

#elevation_function(df, 'lat', 'lon')
#df.head()

In [11]:
## list of point objects
#pointsArr = df.to_records()
#pointsList = []
#for p in pointsArr:
  #  newPoint = (p.lat, p.lon, p.elev_meters)
  #  pointsList.append(newPoint)

In [ ]:

last_element = None
distances = []
for p in pointsList:
    if last_element is not None:
        d = distanceBetweenPoints(last_element, p)
        distances.append(d)
    else:
        distances.append(0)
    last_element = p
df['distance'] = distances
#print(distances)
df.head()

,lat,lon,elev_meters,distance
0,48.6330,-93.9667,341.14,0.000000
1,48.7330,-94.6167,328.80,50.663258
2,45.1947,-93.3257,262.68,410.778145
3,45.1962,-93.2755,272.64,10.713986


In [6]:
def coordinates_to_df(lat, lon):
    # create data frame
    df = pd.DataFrame({
        'lat': lat,
        'lon': lon
    })
    elevation_function(df, 'lat', 'lon')
    pointsArr = df.to_records()
    pointsList = []
    for p in pointsArr:
        newPoint = (p.lat, p.lon, p.elev_meters)
        pointsList.append(newPoint)
    last_element = None
    distances = []
    for p in pointsList:
        if last_element is not None:
            d = distanceBetweenPoints(last_element, p)
            distances.append(d)
        else:
            distances.append(0)
        last_element = p
    df['distance'] = distances
    #print(distances)
    df.head()
    return df
    

In [102]:
coordinates_to_df(latOne, lonOne)

,lat,lon,elev_meters,distance
0,48.6330,-93.9667,341.14,0.000000
1,48.7330,-94.6167,328.80,49137.463438
2,45.1947,-93.3257,262.68,405421.797821
3,45.1962,-93.2755,272.64,3948.164659


In [7]:
import json
def read_segments_to_df(filename):
   list_df = []
   with open(filename) as f:
      segment_data_dict = json.load(f)
      
      #print(segment_data_dict.keys())
      for key in segment_data_dict.keys():
         data = segment_data_dict.get(key)
         lat1 = []
         lon1 = []
         for cord in data:
            lat1.append(cord[0])
            lon1.append(cord[1])
         list_df.append(coordinates_to_df(lat1, lon1))
   return list_df
#print(data)

In [15]:
l = read_segments_to_df('experiment_notebooks/james_kom_detailed_segment_data.json')


FileNotFoundError: [Errno 2] No such file or directory: 'experiment_notebooks/james_kom_detailed_segment_data.json'

In [119]:
for i in l:
    print(i)

          lat        lon  elev_meters    distance
0    34.09464 -117.71377       355.14    0.000000
1    34.09467 -117.71362       355.22   14.236931
2    34.09467 -117.71327       355.12   32.298915
3    34.09476 -117.71181       356.33  135.106672
4    34.09474 -117.71131       356.34   46.194346
..        ...        ...          ...         ...
120  34.09553 -117.71492       357.69   18.949578
121  34.09525 -117.71490       356.67   31.130255
122  34.09482 -117.71493       355.17   47.801181
123  34.09467 -117.71511       354.36   23.524827
124  34.09465 -117.71524       354.10   12.202856

[125 rows x 4 columns]
        lat        lon  elev_meters   distance
0  34.10710 -117.70622       400.70   0.000000
1  34.10708 -117.70530       401.93  84.925089
2  34.10711 -117.70450       402.36  73.891172
         lat        lon  elev_meters   distance
0   34.11110 -117.72251       394.26   0.000000
1   34.11104 -117.72241       394.25  11.376398
2   34.11098 -117.72222       394.26  18.751

In [127]:
j = 0
for i in l:
        print(j)
        filename = "data/Elevation JSON/james_segment_location_data_elevation_%d.json" % j
        j += 1
        with open(filename,"w+") as file:
                lo =  i.to_json()
                file.write(lo)
        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [115]:
d2 = pd.read_json("data/segment_location_data_elevation.json")

ValueError: Trailing data